<a href="https://colab.research.google.com/github/sharonwu827/Deep-Learning/blob/master/Automated_Essay_Scoring_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
#load dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [82]:
# import package
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


!pip install transformers
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Keras functional API
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import Input

import os
import csv
import copy
import nltk
import nltk.tokenize as tk
from sklearn.feature_extraction.text import CountVectorizer
os.chdir('/content/drive/My Drive')
nltk.download('punkt')

import tensorflow as tf
import tensorflow_hub as hub

!pip install pyenchant
!pip install -U pip setuptools wheel
!pip install -U spacy
!sudo apt-get install libenchant1c2a
import enchant #  Enchant spellchecking library
import spacy
import re
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
import urllib.request
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant1c2a is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [83]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/training_set_rel3.tsv",sep='\t', encoding='ISO-8859-1')
dev = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/valid_set.tsv",sep='\t', encoding='ISO-8859-1')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/asap-aes/test_set.tsv",sep='\t', encoding='ISO-8859-1')

In [84]:
prompt = pd.DataFrame({"essay_set":[1,2,3,4,5,6,7,8],
                       "prompt":["More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
                                "Write a persuasive essay to a newspaper reflecting your vies on censorship in libraries. Do you believe that certain materials, such as books, music, movies, magazines, etc., should be removed from the shelves if they are found offensive? Support your position with convincing arguments from your own experience, observations, and/or reading.",
                                "Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
                                "Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.",
                                "Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
                                "Based on the excerpt, describe the obstacles the builders of the Empire State Building faced in attempting to allow dirigibles to dock there. Support your answer with relevant and specific information from the excerpt.",
                                "Write about patience. Being patient means that you are understanding and tolerant. A patient person experience difficulties without complaining.Do only one of the following: write a story about a time when you were patient OR write a story about a time when someone you know was patient OR write a story in your own way about patience.",
                                "We all understand the benefits of laughter. For example, someone once said, “Laughter is the shortest distance between two people.” Many other people believe that laughter is an important part of any relationship. Tell a true story in which laughter was one element or part."]})

## Data processing

In [95]:
def preprocess(df):
  df['normalized_score'] = df['domain1_score'] / df.groupby('essay_set')['domain1_score'].transform('max')

preprocess(train)

In [96]:

train_sents=[]
test_sents=[]

def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

In [97]:
def clean_anonymization(essay):
  '''
  function to remove the anoymaization
  '''
  res=[]
  for i in essay.split():
    if i.startswith("@"):
      continue
    else:
      res.append(i)
  return ' '.join(res)

train['essay']=train['essay'].apply(lambda x:clean_anonymization(x))
# also remove from dev and test
dev['essay']=dev['essay'].apply(lambda x:clean_anonymization(x))
test['essay']=test['essay'].apply(lambda x:clean_anonymization(x))

In [98]:
train.head(3)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6,normalized_score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667
1,2,1,Dear I believe that using computers will benef...,5,4,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750000
2,3,1,"Dear, More and more people use computers, but ...",4,3,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.583333


In [100]:
#-------------- change here------------------#
train = train[:100]
y_train = train['normalized_score']

## BERT - Sentence Embedding

### sentence embedding for semantic score

In [169]:
def bert_tokenized(df):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    segment_ids = []
    attention_masks = []
    for i in range(len(df['essay'])):
        encoded_dict = tokenizer.encode_plus(
            df['essay'][i],  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=512,  # Pad & truncate all sentences.
            pad_to_max_length=True,
            return_attention_mask=True,  # Construct attn. masks.
            return_tensors='pt',  # Return pytorch tensors.
        )

        input_ids.append(encoded_dict['input_ids'])
        segment_ids.append(encoded_dict['token_type_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors (constructed ).
    input_ids = torch.cat(input_ids, dim=0)
    segment_ids = torch.cat(segment_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    return input_ids, segment_ids,attention_masks

In [102]:
 #with torch.no_grad():
       # outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        #hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    #token_embeddings = hidden_states[-1]
    ## Collapsing the tensor into 1-dimension
    #token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    #list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    #return list_token_embeddings

IndentationError: ignored

In [104]:
def bert_embedding(input, mask, segment):
  model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True # Whether the model returns all hidden-states.
                                  )
  # Run the text through BERT, and collect all of the hidden states produced
  # from all 12 layers. 
  with torch.no_grad():
    outputs = model(input_ids = input, attention_mask = mask, token_type_ids = segment)
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2] # The full set of hidden states for this model, stored in the object hidden_statesprint ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
    last_hidden_states = outputs[0]  # get the last hidden state
  return last_hidden_states[:,0,:]

In [171]:
input_ids_ss = bert_tokenized(train)[0]
segment_ids_ss = bert_tokenized(train)[1]
attention_masks_ss  = bert_tokenized(train)[2]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


### sentence embedding for prompt relevant score

In [106]:
train_ps=train.merge(prompt,on='essay_set',how='left')

In [107]:
def generate_input_ids(df):
  encoded_dict = tokenizer.encode_plus(df["prompt"], df["essay"],
                                       max_length=512,  # Pad & truncate all sentences
                                       pad_to_max_length=True,
                                       add_special_tokens = True,
                                       # return_tensors='np'  # Return pytorch tensors.
                                       )
  return encoded_dict['input_ids']

def generate_attention_masks(df):
  encoded_dict = tokenizer.encode_plus(df["prompt"], df["essay"],
                                       max_length=512,  # Pad & truncate all sentences
                                       pad_to_max_length=True,
                                       add_special_tokens = True,
                                       # return_tensors='np'  # Return pytorch tensors.
                                       )
  return encoded_dict['attention_mask']

In [108]:
train_ps['input_ids_ps'] = train_ps.apply(generate_input_ids, axis=1)
train_ps['attention_masks_ps'] = train_ps.apply(generate_attention_masks, axis=1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [109]:
# Search the input_ids for the first instance of the `[SEP]` token.
def generate_segment_ids(df):
  for input_id in train_ps['input_ids_ps']:
    sep_index = []
    sep_index.append(input_id.index(tokenizer.sep_token_id))
    # The number of segment A tokens includes the [SEP] token istelf
    for index in sep_index:
      segment_ids = []
      num_seg_a = index + 1
      # The remainder are segment B
      num_seg_b = len(input_id) - num_seg_a
      # Construct the list of 0s and 1s.
      segment_ids = [0]*num_seg_a + [1]*num_seg_b
  return segment_ids

In [110]:
segment_ids_ps = train_ps.apply(generate_segment_ids, axis=1)

In [111]:
input_ids_ps = train_ps.apply(generate_input_ids, axis=1)
attention_masks_ps = train_ps.apply(generate_attention_masks, axis=1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## LSTM 

In [177]:
# Input data should be an array type, reshape it.
#x_train = np.asarray(input_ids_ss).astype(np.float32)
#y_train = train['normalized_score']

In [165]:
'''
try some number for the hyperparameters need to be tuned later
'''
batch_size = 512
epochs = 8
max_words =  50000
phrase_len = train['essay'].apply(lambda p: len(p.split(' ')))
max_phrase_len = phrase_len.max()

# create the Keras model
lstm = Sequential()
lstm.add(Embedding(input_dim = max_words
                   ,output_dim = 256
                   ,input_length = max_phrase_len))
lstm.add(SpatialDropout1D(0.3))
lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
lstm.add(Dense(256, activation = 'relu'))
lstm.add(Dropout(0.3))

# Continue building the network graph         
lstm.add(Dense(5, activation = 'softmax')) # hidden layer

# Compile the model: before training a model, to configure the learning process, which is done via the compile method. 
lstm.compile(optimizer='rmsprop',loss='mse') # for a mean squared error regression problem, https://faroit.com/keras-docs/1.2.0/getting-started/sequential-model-guide/
print(lstm.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 626, 256)          12800000  
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 626, 256)          0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 1285      
Total params: 13,392,389
Trainable params: 13,392,389
Non-trainable params: 0
_________________________________________

In [184]:
# Train the model
history = lstm.fit(input_ids_ss.numpy(),np.array(y_train),validation_split = 0.1,epochs = 8,batch_size = 512)

Epoch 1/8


TypeError: ignored

In [112]:
# Build a BERT Layer
import tensorflow_hub as hub
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [163]:
model = Sequential()
num_steps=30 
batch_size=20 
hidden_size=500

model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))
model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

NameError: ignored

In [121]:
def build_model(bert_layer, max_len):

    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(5, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [122]:
max_len = 150
train_input = input_ids_ss
train_labels = train["normalized_score"]

In [123]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
keras_layer_4 (KerasLayer)      [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [133]:
# history = model.fit(torch.tensor(input_ids_ps),torch.tensor(y_train),validation_split = 0.1,epochs = 8,batch_size = 512)

ValueError: ignored

In [151]:
inputs = [torch.randn(1, 3) for _ in range(1)]  # make a sequence of length 5

In [150]:
inputs

[tensor([[-1.4881,  0.1021,  0.2828]]),
 tensor([[ 1.7388, -0.2305,  0.9078]]),
 tensor([[ 1.4267, -0.2781,  0.6858]]),
 tensor([[ 0.2146, -0.2458,  0.0310]]),
 tensor([[ 2.1027, -1.3699,  0.3894]])]

In [153]:
# get lstm model
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()
    return model

In [159]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))


regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100


ValueError: ignored

In [155]:
lstm_model.fit(x_train, y_train,batch_size=64, epochs=150)

Epoch 1/150


ValueError: ignored

In [ ]:
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [53]:
def build_model(bert_layer, max_len=512):
  
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]

    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(5, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='mse', metrics=['accuracy'])
    
    return model

In [69]:
# Build model
in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
bert_inputs = [in_id, in_mask, in_segment]

# Instantiate the custom Bert Layer defined above
bert_output = bert_layer(n_fine_tune_layers=10)(bert_inputs)

# Build the rest of the classifier 
dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

ValueError: ignored

## Handcrafted Features

In [ ]:
def get_correct_and_incorrect_spelling(df):
  """
  Function that measures lexical diversity which is the ratio of total words to unique words
  """
  # Load spaCy model
  nlp = spacy.load('en_core_web_sm')
  # load the enchant dictionary
  d = enchant.Dict("en_US")
  
  incorrect_spelling = []
  correct_spelling = []
  
  for essay in df['essay']:
    individual = []
    for word in essay.split():
      individual.append(d.check(word))                                                                         
    output.append((individual.count(False), individual.count(True)))
  return output


def get_lexical_diversity(df):
    """
    Function that measures lexical diversity which is
    The ratio of total words to unique words
    """
    diversity = []
    for essay in df['essay']:
      diversity.append(round(len(tk.word_tokenize(essay)) / float(len(set(tk.word_tokenize(essay)))), 2))
    return diversity


def get_list_of_number_of_pos(df):
    """
    Function that parses the essay for each words POS
    Returns tuples containg for now, nouns, verbs, adverbs and adjectives
    """
    pos = []
    
    for essay in df['essay']:
        parsed_essay = nlp                                                       (essay)
        token_pos = [token.pos_ for token in parsed_essay]
        
        pos.append((token_pos.count('NOUN'), token_pos.count('VERB'), token_pos.count('ADV'), token_pos.count('ADJ')))
        
    return pos